In [ ]:
#Run below to import dependicies
import time
import os
import re
import cobra
from cobra.io import load_matlab_model, read_sbml_model, save_matlab_model, write_sbml_model
from cobra.flux_analysis import flux_variability_analysis
import numpy as np
import pandas as pd
import time
import warnings
import logging

# **Part 1 : Setting File Path**

Please download your data:

1. In your expression file, the column with genes has variosu names: Gene symbols / Gene names / Probe_ids etc.
    
> Rename  the column as **Gene** to process. <br>
File must include only Gene column with other columns shows expression value. Remove unnecessery columns such as ID_REF.

2.   Ensure that the file name and path, including its extension, is entered correctly.

3.   Execute the provided code cells one by one.

Supported file formats  :
    
> For model : .xml .mat <br>
> For expression data : .xlsx .csv



In [ ]:
# Provide paths for your files here.
model_file_path = "C:/Users/Ben/Desktop/GPRmapper/Toolbox/Eflux_data/Case_study/yeast-GEM.mat" 
expression_file_path  = "C:/Users/Ben/Desktop/GPRmapper/Toolbox/Eflux_data/Case_study/normalized.xlsx"


# This code checks if your file paths are correct.
if model_file_path:
    print(f'Model file exists : "{model_file_path}".')
else:
    print(f'Model file does not exists or does not correct.')

if expression_file_path:
    print(f'Model file exists : "{expression_file_path}".')
else:
    print(f'Model file does not exists or does not correct.')

# **Part 2: Processing Model and Expression Data**

To ensure smooth data processing, please follow these steps:

*   Execute the provided code cells sequentially. This approach allows you to monitor the data processing progress effectively.

*   Alternatively, you can streamline the process by selecting a specific code cell, navigating to Runtime > Run after. This action will execute the selected cell and all subsequent cells automatically.

### Reading model and expression data

In [ ]:
# Reading model with cobra

begin_reading = time.time()

is_xml = model_file_path.endswith(".xml")
is_mat = model_file_path.endswith(".mat")
try:
  if is_xml:
    model = cobra.io.read_sbml_model(model_file_path)
    model_backup = cobra.io.read_sbml_model(model_file_path)
    print(f' Model {model_file_path} is successfully read.')
  elif is_mat:
    model = cobra.io.load_matlab_model(model_file_path)
    model_backup = cobra.io.load_matlab_model(model_file_path)
    print(f' Model {model_file_path} is successfully read.')
except:
  print(f' Model {model_file_path} cannot be read. ')

# Reading expression data
is_xlsx = expression_file_path.endswith(".xlsx")
is_csv = expression_file_path.endswith(".csv")
try:
  if is_xlsx:
    expression_data = pd.read_excel(expression_file_path)
    expression_data['Gene'] = expression_data['Gene'].astype(str)
    print(f' Gene expression data {expression_file_path} is successfully read.')
  elif is_csv:
    expression_data = pd.read_csv(expression_file_path)
    expression_data['Gene'] = expression_data['Gene'].astype(str)
    print(f'Gene expression data {expression_file_path} is successfully read.')
except:
  print(f'Gene expression data {expression_file_path} cannot be read. ')

end_reading = time.time()

total_time_reading = end_reading - begin_reading
print(f"Runtime: {int(total_time_reading // 60)} min {int(total_time_reading % 60)} sec")

##  **Part 3 : GPRmapper** 

Returns a file of a reaction expression by calculating models gpr rule.

In [ ]:
# Processing expression data with GPRmapper tool
def parse_and_evaluate(expression):
    while '(' in expression:
        expression = re.sub(r'\(([^()]+)\)', lambda x: "calculated_" + str(calculate_rule(x.group(1))), expression)
    return calculate_rule(expression)

def calculate_rule(rule):
    if 'and' in rule or 'or' in rule:
        or_parts = rule.split('or')
        or_results = []
        for part in or_parts:
            and_elements = part.split('and')
            and_elements = [re.sub(r'[()]', '', element).strip() for element in and_elements]
            and_averages = []
            for element in and_elements:
                if element.startswith('calculated_'):
                    avg_sum = float(element[11:])
                else:
                    filtered_df = expression_data[expression_data['Gene'] == element]
                    if not filtered_df.empty:
                        avg_sum = filtered_df.drop(columns=['Gene']).mean(axis=1).mean()
                    else:
                        avg_sum = 0
                and_averages.append(avg_sum)
            if 'and' in part:
                or_results.append(min(and_averages) if and_averages else 0)
            else:
                or_results.append(sum(and_averages) if and_averages else 0)
        return sum(or_results)
    else:
        filtered_df = expression_data[expression_data['Gene'] == rule]
        if not filtered_df.empty:
            return filtered_df.drop(columns=['Gene']).mean(axis=1).mean()
        return 0

begin_gpr = time.time()

# Extracting reaction rules
reaction_rules = []
for i, reaction in enumerate(model.reactions):
  rule = reaction.gene_reaction_rule
  reaction_rules.append(rule)
GPR_file = [parse_and_evaluate(rule) for rule in reaction_rules]
print('Reaction rules are extracted.')

end_gpr = time.time()

total_time_gpr = end_gpr - begin_gpr
print(f"Runtime: {int(total_time_gpr // 60)} min {int(total_time_gpr % 60)} sec")

In [ ]:
# Saving gpr data to as excel.
GPR_file_df = pd.DataFrame(GPR_file)
GPR_file_df.to_excel("GPRmapper_output.xlsx")

# **Part 4: fE-flux Analysis**

*   **Execution  :**  Run the specified code cell to start the fE-flux analysis.

*   **Logging :** Upon execution, a log file named **"fEflux_process_log.txt"** will be generated and stored in the previously created data folder (refer to Part 1 for the folder details). This log file contains essential details of the analysis process.

*  **Debugging (Optional) :** If you wish to enable debugging for more detailed process insights, please set the designated area to **"True"**.

*  **Re-running Analysis  :** Should you need to conduct the Feasible E-flux analysis again, it is crucial to first re-execute the **"Reading model and expression data"** cell located in Part 2. This step is necessary to ensure the analysis uses the most current data. Be sure to back up the old log file if necessary.

This structured approach is designed to streamline the Feasible E-flux analysis, ensuring both efficiency and accuracy.

In [ ]:
# FVA
begin_fva = time.time()

fva_result = flux_variability_analysis(model)
print('FVA is completed')

# Optimization
fva_result['maximum'] = fva_result['maximum'].apply(lambda x: 0 if x < 0.00001 else x)
fva_result['minimum'] = fva_result['minimum'].apply(lambda x: 0 if x < 0.00001 else x)
fva_result.loc[fva_result['maximum'] < fva_result['minimum'], ['maximum', 'minimum']] = fva_result.loc[fva_result['maximum'] < fva_result['minimum'], ['minimum', 'maximum']].values

max_fluxes = fva_result.maximum.to_list()
min_fluxes = fva_result.minimum.to_list()
print('FVA results are optimized')

end_fva = time.time()

total_time_fva = end_fva - begin_fva
print(f"Runtime: {int(total_time_fva // 60)} min {int(total_time_fva % 60)} sec")

### Running fE-flux

In [ ]:
# Integrating expression data in your model with feasible Eflux method - fEflux tool
begin_eflux = time.time()

# Log file preperation

debugging = False 

logger_name = "fEflux_process_logger"
log_path = "fEflux_process_log.txt"

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger(logger_name)
logger.setLevel(logging.INFO)
logger.propagate = debugging

file_handler = logging.FileHandler(log_path)
file_handler.setLevel(logging.INFO)

log_format = '%(asctime)s - %(message)s'
formatter = logging.Formatter(log_format)
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)

# Performing log2 normalization for GPRmapper output.
normalized_GPR_file = [
    0 if value is None or np.isnan(value) else np.log2(value) if value > 1 else value
    for value in GPR_file
    ]
print('Reaction rules are normalized')

# Initialize counters
all_set = False
process_count, negative_gpr, infeasible_in_lb, infeasible_in_ub, bounds_processed, nan_gpr, bounds_are_zero = 0, 0, 0, 0, 0, 0, 0

print(f'''
Reaction Number is {len(model.reactions)}
Process is starting..
      ''')

# Ignore specific solver infeasibility warnings
warnings.filterwarnings('ignore', category=UserWarning, message="Solver status is 'infeasible'.")
# Take  maximum  value
max_Rxnexp = max(normalized_GPR_file)

time.sleep(1.5)

print("Trying to change all reaction bounds of model at once.")

# Backup Model used for creating all reactions applied with fEflux.
for i, reaction in enumerate(model_backup.reactions):
    if normalized_GPR_file[i] > 0 and pd.isna(normalized_GPR_file[i]) == False:
        reaction.lower_bound = (normalized_GPR_file[i] / max_Rxnexp) * min_fluxes[i]
        reaction.upper_bound = (normalized_GPR_file[i] / max_Rxnexp) * max_fluxes[i]
all_set = model.optimize().status == "feasible"

if not all_set:
    print("Failed. E-flux optimization procedure has started.")
    for i, reaction in enumerate(model.reactions):
        process_count += 1

        if reaction.lower_bound != 0 or reaction.upper_bound != 0:

            if normalized_GPR_file[i] < 0:
                negative_gpr += 1
                logger.info(f"Reaction {i}      : Skipped. GPR value is negative.")
                continue

            if normalized_GPR_file[i] >= 0 and pd.isna(normalized_GPR_file[i]) == False:
                
                try:
                    backup_lb = reaction.lower_bound
                    backup_ub = reaction.upper_bound
                    
                    reaction.lower_bound = (normalized_GPR_file[i] / max_Rxnexp) * min_fluxes[i]
                    reaction.upper_bound = (normalized_GPR_file[i] / max_Rxnexp) * max_fluxes[i] 

                    solution = model.optimize()
                    if solution.status == "feasible":
                        bounds_processed += 1
                        continue
                
                except:   
                    reaction.lower_bound = backup_lb
                    reaction.upper_bound = backup_ub
                
                # Solving LOWER bound
                reaction.lower_bound = (normalized_GPR_file[i] / max_Rxnexp) * min_fluxes[i]

                solution = model.optimize()
                if solution.status == "infeasible":
                    reaction.upper_bound = max_fluxes[i]
                    reaction.lower_bound = min_fluxes[i]
                    

                    infeasible_in_lb += 1
                    logger.info(f"Reaction {i}      : Bounds are assigned as max-min values. Infeasible reaction caused while lower bound change.")
                    continue

                # Solving UPPER bound
                reaction.upper_bound = (normalized_GPR_file[i] / max_Rxnexp) * max_fluxes[i]

                solution = model.optimize()
                if solution.status == "infeasible":
                    reaction.upper_bound = max_fluxes[i]

                    infeasible_in_ub += 1
                    logger.info(f"Reaction {i}      : Upper bound is assigned as max value. Infeasible reaction caused while upper bound change.")
                    continue

                bounds_processed += 1
                continue

            # If there is a NaN value
            else:
                nan_gpr += 1
                logger.info(f"Reaction {i}      : Skipped. GPR value is Nan.")
                continue

        # If both values are 0
        else:
            bounds_are_zero += 1
            logger.info(f"Reaction {i}      : Skipped. Both lower and upper bounds are 0.")
            continue

end_eflux = time.time()

total_time_eflux = end_eflux - begin_eflux
totol_runtime = total_time_eflux +  total_time_fva + total_time_gpr + total_time_reading

if all_set:
    print("Succesfull. The model is feasible with bound values of expression data. Process has completed.")
else:
    print(f"""
> E-flux optimization is completed.
> Log file for details is saved as {log_path}

                    ===   Result overview   ===
Reactions Processed                         :      {process_count}
Total Reactions in Model                    :      {len(model.reactions)}
    (These values should match.)

Bounds Processed                            :      {bounds_processed}
    (Indicates the number of reaction bounds that could feasibly be altered.)

Lower Bound Infeasible                      :      {infeasible_in_lb}
Upper Bound Infeasible                      :      {infeasible_in_ub}
    (These figures represent the number of reactions that became infeasible after changing bound. Bounds were adjusted to maximum and minimum values.)

Negative GPR Associations                   :      {negative_gpr}
Non-Numerical (NaN) GPR Associations        :      {nan_gpr}
    (These scenarios indicate instances where the expression data could not be processed (which is not desired) and were therefore omitted.)

Reactions with Bounds Already Set to Zero   :      {bounds_are_zero}

Process has completed.

Runtime: {int(total_time_eflux // 60)} min {int(total_time_eflux % 60)} sec

Total Runtime:  {int(totol_runtime // 60)}  min {int(totol_runtime % 60)} sec
        """)

In [ ]:
# Saving model as MATLAB
if all_set:
    save_matlab_model(model_backup ,"feasible_model.mat")
else:
    save_matlab_model(model_backup ,"infeasible_model.mat")
    save_matlab_model(model ,"feasible_model.mat")